In [1]:

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np

import logging
import sys
import time


logPath='Logs'
fileName='blackjack_{}'.format(time.time())

logFormatter = logging.Formatter("%(asctime)s [%(threadName)-12.12s] [%(levelname)-5.5s]  %(message)s")
streamFormatter = logging.Formatter("%(asctime)s:  %(message)s")
rootLogger = logging.getLogger()
rootLogger.setLevel(logging.DEBUG)

fileHandler = logging.FileHandler("{0}/{1}.log".format(logPath, fileName))
fileHandler.setFormatter(logFormatter)
rootLogger.addHandler(fileHandler)

consoleHandler = logging.StreamHandler(stream=sys.stdout)
consoleHandler.setFormatter(streamFormatter)
rootLogger.addHandler(consoleHandler)



SEED = 42
NUM_OF_DECKS=1 #8 is max 1 is MIN
NUM_OF_OTHER_PLAYERS=3 #6 is max
SHOE_CUT = 0.75 #1.0 is max
CHIPS = 5_000
TABLE_MIN =100
TABLE_MAX = 1_000
NAME = 'Scott'
PLAYER_POS = 1
STRATEGIES='basic'
np.random.seed(SEED)

In [2]:
class Player:
    '''
    This class is responsible for instantiating a player in the game of blackjack
    
    inputs
    ------
    strategy -> A strategy function the player will use as main strategy in the game
    bank -> Amount of money the player starts with
    name -> name of your player
    
    outputs
    -------
    None
    '''
    def __init__(self, strategy: object, betting_strategy: object, chips: float, name: str):
        #strategy class
        self.strategy = strategy()
        
        #betting class or function
        self.betting_strategy = betting_strategy
        
        #starting money
        self.chips = chips
        
        #flag to check if player is broke
        self.broke = False if chips > 0 else True
        
        #player name
        self.name = name
        
        #keep track of winnings per hand
        self.winnings_per_hand = []
        
        
    def __repr__(self):
        return "{}\nChips:    ${}\nStrategy: {}\n".format(self.name, self.chips, self.strategy)
    
    def _set_table_rules(self, min_, max_):
        
        self.table_min = min_
        self.table_max = max_
        self.betting_strategy = self.betting_strategy(self)
        
    def decision(self, cards):
        return self.strategy._decision(cards)

    
    def bet(self):
        if self.broke: print('Im broke')
        if not self.broke:
            amount = self.betting_strategy.bet()
            
            amount = min(self.chips, amount)
            
            self.chips -= amount
            
            if self.chips <= 0: self.broke = True
                
            return amount
        
        else:
            return -1
        
        
        
class bet_min:
    def __init__(self, Player):
        self.table_min = Player.table_min
    
    def bet(self):
        return self.table_min
    
    def __repr__(self):
        return 'Always bet table minimum'
    
    
    
    

class always_stand:
    def _decision(self, not_used):
        return 'stand'
    
    def __repr__(self):
        return 'Always Stand'
    
    
    
    
class dealer_strategy_stand_on_17:
    
    def _decision(self, cards):

        dealers_cards = cards[0]
        other_players_cards = cards[1]
        
        #check if all players have busted
        totals = np.array([np.sum(x) for x in other_players_cards])
        not_busted = totals[totals < 22]
        if not_busted.size == 0: return 'stand'
        
        
        #follow 'stand on 17 or higher' strategy
        return stand_on_17_or_higher._decision(cards=dealers_cards)
        
        
    def __repr__(self):
        return 'Stand on 17 (Dealer)'
    
    
class stand_on_17_or_higher:
    
    @staticmethod
    def _decision(cards):
        
        total = np.sum(cards)
        
        if total >= 17: return 'stand'
        else: return 'hit'
        
    def __repr__(self):
        return 'Stand on 17 and higher'

In [3]:
'''

These functions should be placed in their own scripts -> Strategies
'''

def basic_strategy(my_cards, dealers_card, players_at_table=None, players_cards=None):

    #separate aces
    aces = my_cards[my_cards==1]
    numbers_and_faces = my_cards[my_cards > 1]
    
    total = np.sum(numbers_and_faces)

    
    #logic if no aces in hand
    if aces.size==0:
        
        #check for 21
        if total == 21: return 'stand'
        
        if total < 16: return 'hit'
        
        if total >= 16: return 'stand'
        
        
    #logic if aces in hand    
    else:
        if 11 + total > 21:
            total += 1
        else:
            total += 11
            
        
        #check for 21
        if total == 21: return 'stand'
        
        if total < 16: return 'hit'
        
        if total >= 16: return 'stand'
        
        
    
    

In [4]:
class Cards:
    def __init__(self, num_of_decks=6):
        assert isinstance(num_of_decks, int), "Number of decks specified must be of type 'int'"
        
        #number of standard 52-card playing decks in the game
        self.num_of_decks = num_of_decks
        
        #aces and number cards
        aces_and_nums = np.array([[x]*4*num_of_decks for x in range(1,11)]).ravel()
        
        #faces
        faces = np.array([10] * 4 * 3 * num_of_decks)
        
        #combine deck
        self.deck = np.concatenate((aces_and_nums, faces))
        
    def __repr__(self):
        return 'Deck of {} cards\n{} cards remaining'.format(52*self.num_of_decks, len(self.deck))
        
    def _shuffle_deck(self):
        
        np.random.shuffle(self.deck)
        
        return self
    
    def _cards_left_percentage(self):
        return len(self.deck) / (52 * self.num_of_decks)
    
    def _deal(self):
        card = self.deck[0].copy()
        self.deck = np.delete(self.deck, 0)
        return card
    
    def _reset_deck(self):
        self.__init__(self.num_of_decks)
        return self._shuffle_deck()

In [5]:
class player_name_generator:
    def __init__(self):
        self.names = ['Frank', 'Jeff', 'Alex', 'Steve', 'John', 'Eddie', 'Brian', 
                      'Richard', 'Nick', 'James', 'Tony','Rhonda', 'Rachel', 
                      'Emily', 'Brianna', 'Peyton', 'Alexis', 'Lisa', 'Jennifer', 'Olivia']
    def _sample_name(self):
        pos = np.random.randint(low=0, high=len(self.names))
        return self.names.pop(pos)


class BlackJack:
    """Blackjack

        Args:
            num_of_decks (int): Number of standard 52 card decks to use at the table.
            num_of_other_players (int): Number of simulated players at the table
            shoe_cut_perc (float): Decimal percentage of deck usage before cards are shuffled again.
                Should be an input of 0.0 <= val <= 1.0

        """
    def __init__(self, num_of_decks, shoe_cut_perc, table_min,table_max, dealer):
        
        assert isinstance(num_of_decks, int), 'num_of_decks parameter must be an integer'
        assert isinstance(shoe_cut_perc, float), 'shoe_cut_perc parameter must be a float with value 0.0 <= x <=1.0'
        assert isinstance(table_min, int), 'min_bet must be an integer'
        assert isinstance(table_max, int), 'max_bet must be an integer'
        assert 0.0 <= shoe_cut_perc <= 1.0, 'shoe_cut_perc must be a float value x with 0.0 <= x <= 1.0'
        assert 1 <= num_of_decks <= 8, 'num_of_decks must be an integer between 1 and 8'
        assert table_min <= table_max, 'min_bet must be less than or equal to max_bet'
        
        #how many cards must be left in the deck before a reshuffle is executed 
        self._shoe_cut_perc = 1 - shoe_cut_perc
        
        #shoe containing all cards of N decks
        self._shoe = Cards(num_of_decks=num_of_decks)._shuffle_deck()
        
        #minmum bet
        self.table_min = table_min
        
        #max bet
        self.table_max = table_max
        
        #player stack
        self.players = []
        
        #flag for my player added
        self.in_the_game = False
        
        self.dealer = dealer
        
    def deal(self):
        
        #check for cut card
        if self._shoe._cards_left_percentage() <= self._shoe_cut_perc:
            logging.info('{}% cards left in the shoe. Reshuffling Deck'.format(int(self._shoe._cards_left_percentage() * 100)))
            self._shoe._reset_deck()
            
        cards = [[] for _ in range(len(self.players))]
        dealers_cards = []
        
        for i in range(2):
            #for each player at the table
            for p in range(len(self.players)):
                cards[p].append(self._shoe._deal())
                
        dealers_cards.append(self._shoe._deal())
                
        return cards , dealers_cards
    
    
    def table_bets(self):
        
        bets = np.empty(shape=(len(self.players), 1))
        
        #bets
        for pos, p in enumerate(self.players):
            bets[pos] = p.bet()
            
        return bets
    
    def hit(self):
        
        return self._shoe._deal()
            
            
            
            
        
    def add_player(self, player, my_player=False):
        '''
        method for adding players to the game
        '''
        #give the player the table betting rules
        player._set_table_rules(min_=self.table_min, max_=self.table_max)
        
        if len(self.players) < 8:
            if my_player and not self.in_the_game:
                self.players.insert(0, player)
                self.in_the_game = True
                self.my_player_pos = 0
                
            elif my_player and self.in_the_game:
                logging.info('Your player is already sitting at the table. No change applied.')
            
            elif not my_player:
                self.players.append(player)
            
            else:
                logging.critical('Fail through on add_player() method. Unknown condition encountered')
            
        else:
            logging.warn('Too many players sitting at the table. Max players allowed: 7')
            
    def _reposition_player(self, pos):
        my_player = self.players.pop(0)
        
        if pos < 0:
            self.players.append(my_player)
            self.my_player_pos = pos
            logging.info('Your player is at the last seat of the table.')
            
        else:
            self.players.insert(pos, my_player)
            self.my_player_pos = pos
            logging.info('Your player is at position {} out of 7 seats at the table'.format(pos))
            
    def summary(self):
        
        for pos, p in enumerate(self.players):
            logging.info('\nSeat: {}\n------------------------\n{}\n************************************\n'.format(pos,p))


            


In [30]:
#instantiate dealer who has infinite chips
dealer = Player(strategy=dealer_strategy_stand_on_17, betting_strategy=None, name='Dealer', chips=np.inf)

In [31]:
#p
table = BlackJack(num_of_decks=NUM_OF_DECKS,
                  shoe_cut_perc=SHOE_CUT,
                  table_min=TABLE_MIN,
                  table_max=TABLE_MAX,
                  dealer=dealer)

In [32]:
player1 = Player(strategy=always_stand,
                 betting_strategy=bet_min,
                 chips=10_000,
                 name='Scott')


player2 = Player(strategy=always_stand,
                 betting_strategy=bet_min,
                 chips=10_000,
                 name='Hank')

In [33]:
table.add_player(player1, my_player=True)
table.add_player(player2, my_player=False)

In [34]:
table.summary()

2020-10-11 01:41:24,241:  
Seat: 0
------------------------
Scott
Chips:    $10000
Strategy: Always Stand

************************************

2020-10-11 01:41:24,243:  
Seat: 1
------------------------
Hank
Chips:    $10000
Strategy: Always Stand

************************************



In [35]:
bets = table.table_bets()

In [58]:
players_cards, dealers_cards = table.deal()

In [59]:
for pos, player in enumerate(table.players):
    player_cards = players_cards[pos]
    other_players_cards = [cards[i] for i in range(len(table.players)) if i != pos]
    
    decision = 'hit'
    while decision is 'hit':
        
        #setup cards as (my cards, everyone elses' cards, dealers cards)
        cards = (player_cards, other_players_cards, dealers_cards)
        decision = player.decision(cards)
        
        
        if decision is 'hit':
            player_cards.append(table.hit())
            
            if np.sum(player_cards) >= 21:
                decision = 'stand'
                

decision = 'hit'
dealers_cards.append(table.hit())
while decision is 'hit':
    cards = (dealers_cards, players_cards, None)
    decision = table.dealer.decision(cards)
    
    if decision is 'hit':
        dealers_cards.append(table.hit())
    
    print(decision)

hit
stand


In [60]:
player_cards.append(table.hit())

In [61]:
players_cards

[[10, 3], [8, 7, 2]]

In [62]:
dealers_cards

[5, 8, 10]

In [54]:
decision

'stand'

In [ ]:
class player_factory:
    def __init__(self, num_of_other_players, strategies, chips, player_pos, name):
        
        assert isinstance(num_of_other_players, int), 'num_of_other_players parameter must be an integer'
        assert 0 <= num_of_other_players <= 6, 'num_of_other_players must be an integer between 0 and 6'
        assert isinstance(chips, (int, list, tuple)), 'Number of chips must be an integer, list, or tuple'
        assert isinstance(player_pos, int), 'player_pos must be an integer value'
        assert isinstance(name, str), 'player name must be a string'
        assert 1 <= player_pos <= num_of_other_players + 1, 'There are {} players at the table, player pos must be 0 <= pos <= {}'.format(num_of_other_players + 1,num_of_other_players + 1)
        assert isinstance(strategies, (str,list,tuple)), 'Strategies must be either {basic, advanced} or of type {list,tuple} containing a strategy for each individual player'

        #number of simulated players
        self._num_of_players = num_of_other_players + 1
        
        #your player position at the table
        self.player_pos = player_pos
        
        #if chips is an int then all players are initialized with the same amount
        if isinstance(chips, int):
            
            #if strategies is just a string then all players are instantiated with the same strategy
            if isinstance(strategies, str):

                #Your player
                main_player = Player(strategy=strategies, name=name, chips=chips)
                
                #manage players list
                self.players = []
                
                #if no other players in the game, just append main player
                if num_of_other_players == 0:
                    self.players.append(main_player)
                    return self
                    
                else:
                    #create a name gen to randomly assign player names
                    name_generator = player_name_generator()

                    #add players to the player list
                    for p in range(num_of_other_players):
                        self.players.append(Player(strategy=strategies, name=name_generator._sample_name(), chips=chips))

                    #insert main player into predefined player_pos
                    self.players.insert(player_pos-1, main_player)
                    return self
            
            #if strategies is a list or tuple then each player will get instantiated with a different value
            #your player's strategy should be first in the list
            else:
                main_player = Player(strategy=strategies[0], name=name, chips=chips)
                
                #manage players list
                self.players = []
                
                #if no other players in the game, just append main player
                if num_of_other_players == 0:
                    self.players.append(main_player)
                    return self
                
                else:
                    #create a name gen to randomly assign player names
                    name_generator = player_name_generator()

                    #add players to the player list
                    for idx,p in enumerate(range(num_of_other_players)):
                        self.players.append(Player(strategy=strategies[idx+1], name=name_generator._sample_name(), chips=chips))

                    #insert main player into predefined player_pos
                    self.players.insert(player_pos-1, main_player)
                    return self
                
                
        #if chips is of type int or list, then each player will get instantiated with a different value
        #your player's chip should be the first element in the {list,tuple}
        elif isinstance(chips, (tuple,list)):
            pass
        
        
        
        

In [8]:
df_wide = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')
df_long=pd.melt(df_wide, id_vars=['Date'], value_vars=['AAPL.Open', 'AAPL.High', 'AAPL.Low', 'AAPL.Close', 'mavg'])

# plotly 

In [9]:
df_long

,Date,variable,value
0,2015-02-17,AAPL.Open,127.489998
1,2015-02-18,AAPL.Open,127.629997
2,2015-02-19,AAPL.Open,128.479996
3,2015-02-20,AAPL.Open,128.619995
4,2015-02-23,AAPL.Open,130.020004
...,...,...,...
2525,2017-02-10,mavg,124.498666
2526,2017-02-13,mavg,125.205166
2527,2017-02-14,mavg,125.953499
2528,2017-02-15,mavg,126.723499


In [7]:
import pandas as pd

2020-11-26 22:48:05,123:  CONFIGDIR=/home/scott/.config/matplotlib
2020-11-26 22:48:05,125:  (private) matplotlib data path: /home/scott/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data
2020-11-26 22:48:05,135:  matplotlib data path: /home/scott/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data
2020-11-26 22:48:05,136:  loaded rc file /home/scott/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/matplotlibrc
2020-11-26 22:48:05,142:  matplotlib version 3.2.1
2020-11-26 22:48:05,144:  interactive is False
2020-11-26 22:48:05,146:  platform is linux
2020-11-26 22:48:05,148:  loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', 'zipimport', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_abc', '_bootlocale', '_locale', 'site', 'os', 'stat', '_stat', 'posixpath', 'genericpath', 

2020-11-26 22:48:05,296:  CACHEDIR=/home/scott/.cache/matplotlib
2020-11-26 22:48:05,306:  Using fontManager instance from /home/scott/.cache/matplotlib/fontlist-v310.json
